## OT or NOT OT

In [ ]:
import os
import signal
import random
from base64 import b64encode
from Crypto.Util.number import getStrongPrime, bytes_to_long
from Crypto.Util.Padding import pad
from Crypto.Cipher import AES
from flag import flag

p = getStrongPrime(1024)

key = os.urandom(32)
iv = os.urandom(AES.block_size)
aes = AES.new(key=key, mode=AES.MODE_CBC, iv=iv)
c = aes.encrypt(pad(flag, AES.block_size))

key = bytes_to_long(key)
print("Encrypted flag: {}".format(b64encode(iv + c).decode()))
print("p = {}".format(p))
print("key.bit_length() = {}".format(key.bit_length()))

signal.alarm(600)
while key > 0:
    r = random.randint(2, p-1)
    s = random.randint(2, p-1)
    t = random.randint(2, p-1)
    print("t = {}".format(t))

    a = int(input("a = ")) % p
    b = int(input("b = ")) % p
    c = int(input("c = ")) % p
    d = int(input("d = ")) % p
    assert all([a > 1 , b > 1 , c > 1 , d > 1])
    assert len(set([a,b,c,d])) == 4

    u = pow(a, r, p) * pow(c, s, p) % p
    v = pow(b, r, p) * pow(c, s, p) % p
    x = u ^ (key & 1)
    y = v ^ ((key >> 1) & 1)
    z = pow(d, r, p) * pow(t, s, p) % p

    key = key >> 2

    print("x = {}".format(x))
    print("y = {}".format(y))
    print("z = {}".format(z))


Ở đây đề bài mã hóa flag bằng AES mode CBC, cho mình ciphertext, iv, một số nguyên tố $p$ 1024 bit, và số bit của key AES

Sau đó đề bài mã hóa từng bit của key từ phải qua trái như sau: 
- gọi $k=k_{n-1}k_{n-2} \cdots k_2k_1k_0$. 
- Với mỗi 2 bit $k_i$ và $k_{i+1}$ (với $i$ chẵn), chọn ngẫu nhiên 3 số $r, s, t$ và xuất ra $t$
- Ta cần nhập vào 4 số khác nhau $a, b, c, d$ trong modulo $p$ và đều lớn hơn 1
- Hệ thống tính $u = (a^r \times c^s) \pmod p$ và $v = (b^r \times c^s) \pmod p$ 
- Tính $x=u \oplus k_i$, $y = v \oplus k_{i+1}$ và $z=(d^r \times t^s) \pmod p$
- Ta nhận được $x$, $y$ và $z$

Ý tưởng: nhận xét từ $z$, $d$ có số mũ như $a$ và $b$, còn $t$ có số mũ như $c$. Như vậy, nếu chọn $d = a \times b \pmod p$ và $c^2 = t \pmod p$ thì ta sẽ có $z = (a \times b)^r \times c^{2s} = (a^r \times c^s) \times (b^r \times c^s) = (u \times v) \pmod p$

Từ đó bruteforce $k_i$ (2 bit), có thể tính được $u = x \oplus k_i$, và từ $v = z \times u^{-1} \pmod p$ và tìm ra $k_{i+1}=y \oplus v$. Mình nhận trường hợp khi $k_{i+1}$ có 1 bit.

Sau 128 vòng mình khôi phục hoàn toàn key và từ đó decrypt được flag.

Mà, quên cái mình nói đi, vì nó **SAI** rồi :)) thực ra là không đúng trong mọi trường hợp

Nếu $c^2 = t \pmod p$ thì $c$ là căn bậc 2 modulo $p$, nhưng nếu $t$ không phải căn bậc 2 modulo $p$ thì sao? 

Thì **không tìm được** $c$ chứ sao :))

Ý tưởng (lần 2, mong là hiệu quả): từ công thức $z$, bình phương 2 vế $z^2 = d^{2s} \times t^{2s} \pmod p$. Mình chọn ngẫu nhiên $a$ và $b$ rồi tìm $d$ thỏa mãn $a \times b = d^2 \pmod p$ và $c=t$. Như vậy mình vẫn có $z^2 = u \times v \pmod p$. Việc này là khả thi vì ở "ý tưởng" trước, $t$ là từ hệ thống cho, còn ở đây $a, b, c, d$ là do mình chọn

Như vậy ý tưởng được viết lại như sau:
- Chọn ngẫu nhiên $a$ và $b$, tìm $d$ thỏa mãn $d^2 = a \times b \pmod p$ và chọn $c=t$. Lặp lại tới khi thỏa điều kiện của 4 biến này
- $z^2 = u \times v \pmod p$
- bruteforce $k_i$ (2 bit), từ đó tính $u = x \oplus k_i$, sau đó $v = z^2 \times u^{-1} \pmod p$ và tìm $k_{i+1}=y \oplus v$. Nhận trường hợp $k_{i+1}$ có 1 bit

Sau 128 vòng thì mình có key và từ đó tìm decrypt được flag


In [ ]:
from pwn import *
from base64 import b64decode
from Crypto.Util.number import bytes_to_long, long_to_bytes
import random
from Crypto.Cipher import AES

def legendre_symbol(a, p):
	ls = pow(a, (p-1)//2, p)
	if ls == p-1: 
		return -1
	return 1
    
def modular_sqrt(a, p):
	if legendre_symbol(a, p) == -1:
		return 0
	elif a == 0:
		return 0
	elif p == 2:
		return 0
	elif p % 4 == 3:
		return pow(a, (p+1)//4, p)
	u = p-1
	s = 0
	while u % 2 == 0:
		u = u // 2
		s += 1
	z = 2
	while legendre_symbol(z, p) != -1:
		z += 1
	c = pow(z, u, p)
	t = pow(a, u, p)
	r = pow(a, (u+1) // 2, p)
	while True:
		y = t
		i = 0
		for i in range(s):
			if y == 1:
				break
			y = pow(y, 2, p)
		if i == 0:
			return r
		b = pow(c, 2**(s-i-1), p)
		s = i
		c = (b*b) % p
		t = (t*b*b) % p
		r = (r*b) % p

sock = remote('crypto.ctf.zer0pts.com', 10130)
enc = sock.recvline().strip()[16:]
p = int(sock.recvline().strip()[4:])
bitlen = int(sock.recvline().strip()[-3:])
key = ""
for _ in range(0, bitlen, 2):
    t = int(sock.recvline().strip()[4:])
    while True:
        a = random.randint(2, p-1)
        b = random.randint(2, p-1)
        c = t
        d = modular_sqrt(a * b, p)
        if len(set([a, b, c, d])) == 4 and all([a > 1, b > 1, c > 1, d > 1]):
            break
    sock.recv(1024)
    sock.sendline(str(a).encode())
    sock.recv(1024)
    sock.sendline(str(b).encode())
    sock.recv(1024)
    sock.sendline(str(c).encode())
    sock.recv(1024)
    sock.sendline(str(d).encode())
    x = int(sock.recvline().strip()[4:])
    y = int(sock.recvline().strip()[4:])
    z = int(sock.recvline().strip()[4:])
    z = pow(z, 2, p)
    for key1 in range(2):
        u = x ^ key1
        v = (z * pow(u, -1, p)) % p
        key2 = y ^ v
        if key2 <= 1:
            key = str(key2) + str(key1) + key
            break
    # print(key)

sock.close()
enc = b64decode(enc)
iv, cipher = enc[:16], enc[16:]
print(iv, cipher, key)
key = long_to_bytes(int(key, 2))
# assert len(key) == AES.block_size
aes = AES.new(key, AES.MODE_CBC, iv)
print(aes.decrypt(cipher))

# b'zer0pts{H41131uj4h_H41131uj4h}\x02\x02'

**Unintended idea** từ anh **bmtd**

Chọn random $a$, chọn $b=a^{-1} \pmod p$, $c=-1$ và $d$ tùy ý. Khi đó ta có $$u \times v = a \times a^{-1} \times (-1)^{2s} = 1 \pmod p$$. Như vậy mình bruteforce $key_i$ và tìm được $u = x \oplus key_i$ thì với $v$ là nghịch đảo của $u$ modulo $p$, mình tính được $v=u^{-1} \pmod p$, suy ra $y$ rồi $key_{i+1}$